In [2]:
!pip install lingualytics

In [3]:
print("Pavan")

Pavan


In [4]:
!pip install pandas

In [5]:
import numpy as np

In [6]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits
import pandas as pd

In [63]:
DATASET = "E:\Bobby\pk.xlsx"

In [66]:
df=pd.read_excel(DATASET)
df

,Q,A
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...
1,Have feedback or want to know more?,We are a passionate set of application focused...
2,What components does kandi cover?,kandi helps you select software components acr...
3,How do I use kandi?,kandi provides two simplified experiences to h...
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...
5,How do I implement the components that I have ...,The component listing and detailed insights pa...


In [70]:
df['clean_Q'] = df['Q'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 

C:\Users\Pavan Sai.K\anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


In [71]:
df

,Q,A,clean_Q
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [72]:
!pip install -U sentence-transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=2f25defa0712392d333b0f304732b4ecc8b863054d8d941b202bbe280e790ab4
  Stored in directory: c:\users\pavan sai.k\appdata\local\pip\cache\wheels\52\19\88\6625593382e23a926740e6fcee0f2df0a0de25766094842a28
Successfully built sentence-transformers


In [74]:
!pip show sentence-transformers

Name: sentence-transformers
Version: 2.1.0
Summary: Sentence Embeddings using BERT / RoBERTa / XLM-R
Home-page: https://github.com/UKPLab/sentence-transformers
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: c:\users\pavan sai.k\anaconda3\lib\site-packages
Requires: scikit-learn, huggingface-hub, sentencepiece, tqdm, numpy, tokenizers, torchvision, scipy, transformers, nltk, torch
Required-by: 


In [75]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [76]:
q_embs = model.encode(df['clean_Q'], convert_to_tensor=True)

C:\Users\Pavan Sai.K\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [77]:
q_embs.shape

torch.Size([6, 384])

In [78]:
usr_query = "tell me about kandi"

In [79]:
df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
df_usr_query

,usr_query
0,tell me about kandi


In [80]:
df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
df_usr_query

C:\Users\Pavan Sai.K\anaconda3\lib\site-packages\lingualytics\preprocessing.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  return s.str.replace(rf"([{punctuation}])+", " ")


,usr_query,clean_usr_Q
0,tell me about kandi,tell me about kandi


In [81]:
usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
usr_q_emb.shape

C:\Users\Pavan Sai.K\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


torch.Size([1, 384])

In [82]:
import torch

In [83]:
cos_fn = torch.nn.CosineSimilarity(dim=1,eps=1e-6)

In [86]:
idx = np.argmax(cos_fn(usr_q_emb,q_embs)).item()

In [87]:
df['A'][idx]

'kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [88]:
def pred_answer(usr_query):
    df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
    df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
    usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1,eps=1e-6)
    idx = np.argmax(cos_fn(usr_q_emb,q_embs)).item()
    return df['A'][idx]

In [90]:
while True:
    usr_query = input("Ask a query: ")
    if usr_query == "exit":
        break
    else:
        print(pred_answer(usr_query))
        print("-----------------------------------------------")

Ask a query: tell me about kandi
kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.
-----------------------------------------------
Ask a query: I want to know more
We are a passionate set of application focused techies. We’d love to hear from you on your feedback, questions, and any other comments.
Direct Message us on Twitter Message @OpenWeaverInc
You can email us at kandi.support@openweaver.com
Join our Discord community here
-----------------------------------------------
Ask a query: exit
